In [293]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
import keras
from keras import Input
from keras.layers import Dense
from keras.models import Model
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
demo = pd.read_csv("/content/gdrive/My Drive/Smart India Hackathon/THE DATASET/Demographics.csv")
case = pd.read_csv("/content/gdrive/My Drive/Smart India Hackathon/THE DATASET/Case_Documents.csv")

In [334]:
case.head()

,CaseId,DocName,DocType,DateOfService
0,5795,5795_0,OUTBOUND PHYS NOTE,2018-10-26 19:53:39
1,5795,5795_1,EDPHYNOTES,2018-10-26 19:54:00
2,5795,5795_2,OUTBOUND PHYS NOTE,2018-10-26 20:32:00
3,5795,5795_3,EDPHYNOTES,2018-10-26 20:33:00
4,5795,5795_4,NURSE,2018-10-26 21:47:43


In [0]:
df = pd.read_csv("/content/gdrive/My Drive/Smart India Hackathon/THE DATASET/Demographics.csv")

In [0]:
#demographics preprocessing
to_datetime = ['DateOfBirth', 'DateOfAdmission', 'DateOfDischarge']

for feature in to_datetime:
  demo[feature] = pd.to_datetime(demo[feature], format = '%Y/%m/%d')
  
demo['DurationOfStay'] = demo['DateOfDischarge'] - demo['DateOfAdmission']
demo['Age'] = demo['DateOfAdmission'] - demo.DateOfBirth
demo['Age'] = np.array([[demo['Age'][i].days/365]for i in range(demo.shape[0])])
demo['Age'] = pd.to_numeric(demo['Age'])
demo['Age'] = np.ceil(demo['Age'])
demo.drop(labels= ['DateOfDischarge', 'DateOfAdmission'], axis = 1, inplace= True)
#demo = pd.get_dummies(demo, columns= ['DischargeDisposition', 'ServiceLine', 'PatientClass', 'Gender'])
demo.drop(labels= 'DateOfBirth', inplace= True, axis = 1)
demo['DurationOfStay'] = np.array([[demo['DurationOfStay'][i].seconds]for i in range(demo.shape[0])])


In [0]:
demo = pd.get_dummies(demo, columns= ['DischargeDisposition', 'ServiceLine', 'PatientClass', 'Gender'])

In [337]:
case.head()

,CaseId,DocName,DocType,DateOfService
0,5795,5795_0,OUTBOUND PHYS NOTE,2018-10-26 19:53:39
1,5795,5795_1,EDPHYNOTES,2018-10-26 19:54:00
2,5795,5795_2,OUTBOUND PHYS NOTE,2018-10-26 20:32:00
3,5795,5795_3,EDPHYNOTES,2018-10-26 20:33:00
4,5795,5795_4,NURSE,2018-10-26 21:47:43


In [0]:
uni = case.DocType.unique()
for new in uni:
  demo[new] = np.zeros(demo.shape[0])

In [0]:
cases = case.CaseId.unique()

In [0]:
list(case[case.CaseId == 5795].DocType.unique())

['OUTBOUND PHYS NOTE', 'EDPHYNOTES', 'NURSE', 'EDNURNOTES']

In [0]:
docType = {}
for new in cases:
  unique = case[case.CaseId == new].DocType.unique()
  docType[new] = list(unique)

In [302]:
demo.head()

,CaseId,Gender,PatientClass,ServiceLine,DischargeDisposition,DurationOfStay,Age,OUTBOUND PHYS NOTE,EDPHYNOTES,NURSE,...,FOOT LT2,PD.HYPERK,CC/ANG/STE,FIST/SINUS,PD.SEV.SEP,HEELRIGHT,UAE,MRAHEADW/,PEDSTUH&P,MANDIBLE4V
0,0,F,Emergency,Default,DISCH TO HOME,960,54.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,F,Emergency,Default,DISCH TO HOME,14160,33.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,M,Emergency,Default,AHR:DISCH TO HOME,15660,17.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,F,Emergency,Default,AHR:DISCH TO HOME,2880,29.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,M,Emergency,Default,DISCH TO HOME,3420,16.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
ind = demo[demo.CaseId == 1].index

In [0]:
demo.loc[ind, ['FIST/SINUS', 'ServiceLine']]

,FIST/SINUS,ServiceLine
5931,0.0,EMERGENCY ROOM OBSERVATION


In [0]:
len(cases)

9683

In [0]:
np.ones(4).shape

(4,)

In [0]:
for new in cases:
  get = docType[new]
  ind = demo[demo.CaseId == new].index
  for feat in get: 
    demo.loc[ind, feat] = 1

In [0]:
demo.to_csv('/content/gdrive/My Drive/Demo_processed.csv')

In [0]:
demo.drop(labels= 'CaseId', inplace= True, axis = 1) ##dont remove now

In [332]:
demo.drop(labels= a, inplace= True, axis = 1)

ValueError: ignored

In [0]:
#normalizing the dataset
ss = StandardScaler()
ss.fit(demo.values)
X = ss.transform(demo.values)

In [0]:
def enc_dec(input_shape):
  
  input_ = Input(shape=(513,))
  encoded = Dense(1024, activation='relu')(input_)
  encoded = Dense(512, activation='relu')(encoded)
  encoded = Dense(256, activation='relu')(encoded)
  encoded = Dense(512, activation='sigmoid')(encoded)

  decoded = Dense(256, activation='relu')(encoded)
  decoded = Dense(512, activation='relu')(decoded)
  decoded = Dense(512, activation='relu')(decoded)
  decoded = Dense(513)(decoded)

  autoencoder = Model(input_, decoded)
  encoder = Model(input_, encoded)
  
  return autoencoder, encoder

In [0]:
input_shape = (X.shape[1],)
autoencoder, encoder = enc_dec(input_shape)

In [0]:
autoencoder.compile(loss=  'mse', optimizer = 'adam', metrics = ['accuracy'])

In [316]:
train_history = autoencoder.fit(X, X, epochs=100, batch_size=512, validation_split= 0.1)

Train on 8714 samples, validate on 969 samples
Epoch 1/100
8714/8714 [==============================] - 0s 55us/step - loss: 0.0771 - acc: 0.8624 - val_loss: 2.9560 - val_acc: 0.4169
Epoch 2/100
8714/8714 [==============================] - 0s 56us/step - loss: 0.0781 - acc: 0.8461 - val_loss: 2.9908 - val_acc: 0.4014
Epoch 3/100
8714/8714 [==============================] - 0s 56us/step - loss: 0.0877 - acc: 0.8529 - val_loss: 2.9609 - val_acc: 0.3922
Epoch 4/100
8714/8714 [==============================] - 0s 55us/step - loss: 0.0835 - acc: 0.8185 - val_loss: 2.9822 - val_acc: 0.4200
Epoch 5/100
8714/8714 [==============================] - 0s 55us/step - loss: 0.0747 - acc: 0.8400 - val_loss: 2.9067 - val_acc: 0.4592
Epoch 6/100
8714/8714 [==============================] - 0s 53us/step - loss: 0.1019 - acc: 0.8399 - val_loss: 2.9078 - val_acc: 0.4211
Epoch 7/100
8714/8714 [==============================] - 0s 52us/step - loss: 0.0846 - acc: 0.8431 - val_loss: 2.9590 - val_acc: 0.4118
E

In [0]:
enc = encoder.predict(X)
sim = cosine_similarity(enc, enc)
X_dec = autoencoder.predict(X)

In [0]:
import pickle
import pandas as pd
import csv
import time
import datetime
import os
import glob

class Patient:
  def fetchDocuments(self):
    column_names = []
    documents = []
    f = open( "/content/gdrive/My Drive/Smart India Hackathon/THE DATASET/Case_Documents.csv" , "r" )
    csvFile = csv.reader( f )
    for row in csvFile:
      if len( column_names ) == 0:
        column_names = row
        continue
      if int(row[0]) == int(self.caseID):
        documents.append( {
          'CaseId' : int( row[0] ) ,
          'DocName' : row[1] ,
          'DocType' : row[2] ,
          'DateOfService' : datetime.datetime.strptime( row[3] , "%Y-%m-%d %H:%M:%S" ),
          'PhysicianID' : int( row[4] ),
          'Speciality' : row[5]
        } )

    f.close()

    return documents
  
  def combineDocuments(self):
    ret = ""
    for item in self.documents:
      ret = ret + str( item['DocType'] ) + " "
      f = open( "/content/gdrive/My Drive/Smart India Hackathon/data/Raw_Texts/temp/"+item['DocName'] , "r" )
      rd = f.read()
      rd = rd.split("\n")
      ret = ret + ( " ".join( rd ) ).strip() + " "
      f.close()
    return ret

  def combineDocumentsModified(self):
    ret = ""
    iden = str( self.documents[0]['DocName'] )[:6]
    files = []
    for file in glob.glob("/content/gdrive/My Drive/Smart India Hackathon/data/Raw_Texts/temp/*.*"):
      if iden in file:
        files.append( file )
    for file in files:
      print( file )
      f = open( file , "r" )
      rd = f.read()
      rd = rd.split("\n")
      ret = ret + " ".join( rd ) + " "
    return ret.strip()
    
  def fetchSpecialities(self):
    ret = set()
    for item in self.documents:
      ret.add( str(item['Speciality']).lower() )
    return ret

  def documentContentDict(self):
    ret = []
    for item in self.documents:
      f = open( "/content/gdrive/My Drive/Smart India Hackathon/data/Raw_Texts/temp/"+item['DocName'] , "r" )
      rd = f.read()
      rd = rd.split("\n")
      ret.append( { 'DocType' : item['DocType'] , 'DocContent' : ( " ".join( rd ) ).strip() } )
      f.close()
    return ret

  def documentContentDictModified(self):
    ret = []
    iden = str( self.documents[0]['DocName'] )[:6]
    files = []
    for file in glob.glob("/content/gdrive/My Drive/Smart India Hackathon/data/Raw_Texts/temp/*.*"):
      if iden in file:
        files.append( file )
    for file in files:
      f = open( file , "r" )
      rd = f.read()
      rd = rd.split("\n")
      ret.append( ( " ".join( rd ) ).strip() )
    return ret

  def fetchXmlDocumentFnamesModified(self):
    ret = []
    iden = str( self.documents[0]['DocName'] )[:6]
    files = []
    for file in glob.glob("/content/gdrive/My Drive/Smart India Hackathon/data/NER_XML/*.xml"):
      if iden in file:
        files.append( file )
    return files
  
  def __init__(self, caseID, dob, gender, dateOfAdmission, dateOfDischarge, patientClass, statusID, serviceLine, payerID, dischargeDisposition, pointOfCare):
    self.caseID = caseID
    self.dob = dob
    self.gender = gender
    self.dateOfAdmission = dateOfAdmission
    self.dateOfDischarge = dateOfDischarge
    self.patientClass = patientClass
    self.statusID = statusID
    self.serviceLine = serviceLine
    self.payerID = payerID
    self.dischargeDisposition = dischargeDisposition
    self.pointOfCare = pointOfCare
    self.documents = self.fetchDocuments()
    self.age = int( ( self.dob - datetime.datetime.now() ).days / 365.25 )
    self.duration = int( ( self.dateOfDischarge - self.dateOfAdmission ).days )
    self.specialities = self.fetchSpecialities()


In [0]:
from PatientRecord import *
fileName = 'mGD'
fileObject = open('/content/gdrive/My Drive/' + fileName, 'rb')
db = pickle.load(fileObject)    
x = db['Patients']
fileObject.close()

In [0]:
doc = []
for i in range(len(x)):
  new = x[i].documentContentList
  for instance in new:
    doc.append(instance)

In [154]:
len(doc)

54765

In [201]:
mapp[6]

0

In [0]:
d = 0
ax = []
for count in mapp:
  if(count == 0):
    pass
  else:
    for i in range(count):
      ax.append(d)
  d += 1

In [208]:
len(ax)

54765

In [0]:
for i in range(0):
  print('5')

In [130]:
mapp

[2,
 3,
 4,
 4,
 3,
 3,
 0,
 5,
 4,
 4,
 3,
 2,
 9,
 5,
 9,
 2,
 0,
 3,
 1,
 2,
 6,
 2,
 4,
 4,
 6,
 6,
 2,
 4,
 1,
 2,
 4,
 2,
 6,
 2,
 5,
 2,
 2,
 7,
 2,
 5,
 3,
 12,
 0,
 2,
 5,
 1,
 2,
 2,
 1,
 4,
 5,
 2,
 5,
 2,
 3,
 7,
 2,
 3,
 2,
 2,
 0,
 1,
 4,
 2,
 6,
 4,
 2,
 4,
 3,
 5,
 3,
 7,
 1,
 2,
 2,
 4,
 3,
 4,
 2,
 3,
 2,
 4,
 16,
 4,
 4,
 5,
 4,
 5,
 1,
 2,
 2,
 4,
 2,
 11,
 2,
 4,
 3,
 4,
 16,
 2,
 5,
 8,
 2,
 0,
 2,
 1,
 2,
 2,
 7,
 2,
 2,
 1,
 5,
 4,
 3,
 1,
 2,
 1,
 4,
 4,
 1,
 3,
 2,
 1,
 2,
 5,
 6,
 5,
 2,
 7,
 4,
 2,
 5,
 2,
 1,
 3,
 4,
 2,
 2,
 2,
 3,
 2,
 4,
 6,
 7,
 2,
 4,
 6,
 4,
 2,
 4,
 2,
 3,
 8,
 3,
 3,
 5,
 2,
 12,
 2,
 4,
 4,
 4,
 0,
 6,
 2,
 4,
 3,
 7,
 5,
 3,
 5,
 5,
 7,
 7,
 2,
 2,
 3,
 2,
 3,
 5,
 3,
 5,
 7,
 4,
 2,
 2,
 6,
 3,
 5,
 4,
 2,
 4,
 2,
 7,
 6,
 2,
 2,
 12,
 5,
 5,
 4,
 4,
 2,
 1,
 3,
 4,
 2,
 3,
 2,
 3,
 2,
 6,
 6,
 4,
 6,
 5,
 4,
 2,
 2,
 2,
 2,
 2,
 6,
 3,
 2,
 6,
 3,
 5,
 1,
 2,
 1,
 4,
 6,
 5,
 3,
 0,
 2,
 6,
 5,
 2,
 2,
 3,
 2,
 4,
 3,
 3,
 4,
 5

In [0]:
mapp = []
for i in range(len(x)):
  mapp.append(len(x[i].documentContentList))

In [0]:
embedd_df = pd.read_table('/content/gdrive/My Drive/Smart India Hackathon/THE DATASET/glove.6B.50d.txt', sep= ' ', index_col= 0,  header= None, quoting= csv.QUOTE_NONE)

In [0]:
embedd_df.drop(labels= 14798, inplace = True, axis = 0)
words_glove = embedd_df.index

In [0]:
def word_to_vec(word):
    return np.array(embedd_df.loc[word]).reshape((50, ))

In [28]:
np.zeros((50, )).shape

(50,)

In [0]:
docs = []

In [0]:
i = 0
vocab = {}
voc = []
for instance in doc:
  for word in instance:
     voc.append(word)
uni = sorted(set(voc))
for word in uni:
  vocab[word] = i
  i += 1

In [0]:
docs = []
for instance in doc:
  new = []
  for words in instance:
    new.append(words)
  docs.append(new)

In [60]:
max(map(lambda x: len(x), docs))

1602

In [0]:
PAD = '#PAD'

In [0]:
vocab[PAD] = 160535

In [0]:
X_ind = np.zeros((len(docs), 1000)) + vocab[PAD]

In [0]:
def sent_to_ind(sentence, vocab):
  ret = np.zeros((1, 1000)) + vocab[PAD]
  if(len(sentence) >= 1000):
    for i in range(1000):
      ret[:, i] = vocab[sentence[i]]
  else:
    for i in range(len(sentence)):
      ret[:,i] = vocab[sentence[i]]
    
  return ret

In [0]:
for i in range(len(X_ind)):
  X_ind[i,:] = sent_to_ind(docs[i], vocab)

In [0]:
from keras.models import Model
from keras.layers import Embedding
import keras
from keras import Input

In [0]:
from keras.layers import Dense

In [0]:
vocab_length = len(vocab)

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
ss = StandardScaler()
ss.fit(X_ind)
X_ind_norm = ss.transform(X_ind)

In [0]:
def enc_dec(input_shape, vocab_length, emb_dim = 50, max_len = 1000):
  sentence_indices = Input(input_shape)
  encode = keras.layers.Dense(units = 950)(sentence_indices)
  encode = keras.layers.Dense(units = 900)(encode)
  decode = keras.layers.Dense(units = 950)(encode)
  decode = keras.layers.Dense(units = 1000)(decode)
  
  model = Model(input= sentence_indices, output = decode)
  encoder = Model(input = sentence_indices, output = encode)
  
  return encoder, model

In [279]:
input_shape = (1000,)
encoder, model = enc_dec(input_shape, vocab_length)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [0]:
model.compile(loss = 'mse', optimizer = 'adam', metrics = ['acc'])

In [281]:
model.fit(X_ind_norm, X_ind_norm, batch_size = 512, epochs = 100, validation_split = 0.1)

Train on 49288 samples, validate on 5477 samples
Epoch 1/100
49288/49288 [==============================] - 2s 48us/step - loss: 0.5030 - acc: 0.0357 - val_loss: 0.3513 - val_acc: 0.1455
Epoch 2/100
49288/49288 [==============================] - 2s 36us/step - loss: 0.2328 - acc: 0.3238 - val_loss: 0.2430 - val_acc: 0.4559
Epoch 3/100
49288/49288 [==============================] - 2s 36us/step - loss: 0.1808 - acc: 0.5414 - val_loss: 0.2218 - val_acc: 0.5666
Epoch 4/100
49288/49288 [==============================] - 2s 36us/step - loss: 0.1552 - acc: 0.6400 - val_loss: 0.2110 - val_acc: 0.6593
Epoch 5/100
49288/49288 [==============================] - 2s 36us/step - loss: 0.1389 - acc: 0.6836 - val_loss: 0.1838 - val_acc: 0.7108
Epoch 6/100
49288/49288 [==============================] - 2s 36us/step - loss: 0.1366 - acc: 0.7096 - val_loss: 0.1906 - val_acc: 0.7395
Epoch 7/100
49288/49288 [==============================] - 2s 36us/step - loss: 0.1251 - acc: 0.7193 - val_loss: 0.1906 - v

In [0]:
encoded = encoder.predict(X_ind_norm)

In [283]:
encoded.shape

(54765, 900)

In [0]:
data = pd.DataFrame(encoded)

In [214]:
encoded.shape

(54765, 256)

In [0]:
data['CaseId'] = np.array(ax)

In [286]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,891,892,893,894,895,896,897,898,899,CaseId
0,0.537230,0.862290,-1.873075,-3.151354,0.204930,-1.970290,-1.085211,-1.501734,-0.837636,1.391199,...,1.022847,-0.561760,-0.661533,-0.052471,2.021026,0.454799,-0.949964,0.785020,1.324450,0
1,-0.551123,1.102552,-1.781457,-3.259537,-1.322150,-1.251174,-1.472921,-1.170985,-0.312611,0.837767,...,0.892323,-0.045914,-0.448795,0.005537,2.593867,0.239680,0.215307,0.277375,0.752945,0
2,0.280744,1.815632,0.037692,-0.481907,1.083172,1.005171,0.767590,0.047525,0.073320,-1.754606,...,-0.152583,0.040455,-0.632800,-0.013239,1.354242,1.314673,0.565169,-1.206864,-0.035955,1
3,-0.534303,-0.001208,-2.106359,-3.420665,0.137405,-1.939584,-0.795812,-1.935252,0.055148,0.745451,...,0.905829,0.505964,-0.646065,-0.359414,1.082168,1.126747,-0.107244,0.089846,1.641471,1
4,-1.380827,1.120205,-1.450355,-2.683599,-1.489475,-1.072515,-1.240349,-1.736282,-0.648207,1.083281,...,1.764081,-0.315962,-0.521650,0.299854,1.190223,0.291317,1.554775,0.703956,0.935607,1


ddd

In [0]:
new_data = data.groupby('CaseId').mean()
new_data['CaseId'] = new_data.index

In [0]:
new_data_val = new_data.values

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

In [318]:
encoded.shape

(54765, 900)

In [319]:
enc.shape

(9683, 512)

In [0]:
 case_id = new_data.CaseId

In [0]:
set_1 = set(range(enc.shape[0]))
set_2 = set(new_data.CaseId.unique())

In [0]:
a  = set_1 - set_2

In [0]:
a = list(a)

In [0]:
enc_removed = np.delete(enc, a, axis=0)

In [345]:
enc_removed.shape

(9408, 512)

In [0]:
new_data_values = new_data.values

In [0]:
all_enc = np.concatenate((enc_removed, new_data_values), axis = 1)

In [352]:
all_enc.shape

(9408, 1413)

In [0]:
ss = StandardScaler()
ss.fit(all_enc)
all_enc_norm = ss.transform(all_enc)

In [0]:
def all_enc_dec(input_shape):
  
  input_ = Input(shape=(1413,))
  encoded = Dense(1054, activation='relu')(input_)
  encoded = Dense(1000, activation='relu')(encoded)
  encoded = Dense(900, activation='relu')(encoded)
  encoded = Dense(890, activation='sigmoid')(encoded)

  decoded = Dense(900, activation='relu')(encoded)
  decoded = Dense(1000, activation='relu')(decoded)
  decoded = Dense(1054, activation='relu')(decoded)
  decoded = Dense(1413)(decoded)

  autoencoder = Model(input_, decoded)
  encoder = Model(input_, encoded)
  
  return autoencoder, encoder

In [0]:
auto_perm_enc, all_enc_perm = all_enc_dec((1413,))

In [0]:
auto_perm_enc.compile(loss = 'mse', optimizer = 'adam', metrics = ['acc'])

In [407]:
auto_perm_enc.fit(all_enc_norm, all_enc_norm, epochs = 200, batch_size = 2048)

Epoch 1/200
9408/9408 [==============================] - 1s 60us/step - loss: 0.0591 - acc: 0.5805
Epoch 2/200
9408/9408 [==============================] - 0s 53us/step - loss: 0.0596 - acc: 0.5782
Epoch 3/200
9408/9408 [==============================] - 0s 51us/step - loss: 0.0611 - acc: 0.5750
Epoch 4/200
9408/9408 [==============================] - 0s 49us/step - loss: 0.0606 - acc: 0.5768
Epoch 5/200
9408/9408 [==============================] - 0s 51us/step - loss: 0.0601 - acc: 0.5774
Epoch 6/200
9408/9408 [==============================] - 0s 49us/step - loss: 0.0596 - acc: 0.5809
Epoch 7/200
9408/9408 [==============================] - 0s 50us/step - loss: 0.0593 - acc: 0.5762
Epoch 8/200
9408/9408 [==============================] - 0s 49us/step - loss: 0.0595 - acc: 0.5756
Epoch 9/200
9408/9408 [==============================] - 0s 50us/step - loss: 0.0592 - acc: 0.5779
Epoch 10/200
9408/9408 [==============================] - 0s 49us/step - loss: 0.0589 - acc: 0.5804
Epoch 11/

In [0]:
new_enc = all_enc_perm.predict(all_enc_norm)

In [0]:
sim = cosine_similarity(new_enc, new_enc)

In [0]:
similarity = pd.DataFrame(sim)

In [0]:
similarity.index = case_id

In [417]:
similarity.head()

,0,1,2,3,4,5,6,7,8,9,...,9398,9399,9400,9401,9402,9403,9404,9405,9406,9407
CaseId,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.588883,0.563739,0.562836,0.356061,0.522536,0.378793,0.449101,0.322041,0.509015,...,0.424485,0.409977,0.462134,0.457137,0.390549,0.506623,0.308850,0.350424,0.267520,0.383878
1,0.588883,1.000000,0.516514,0.605817,0.343116,0.486205,0.357754,0.482550,0.397628,0.621465,...,0.393431,0.403627,0.573512,0.559287,0.377774,0.575717,0.403475,0.359243,0.347360,0.402865
2,0.563739,0.516514,1.000000,0.716051,0.393760,0.443990,0.384367,0.482685,0.314629,0.520678,...,0.334657,0.332119,0.426567,0.433672,0.314837,0.478888,0.288216,0.290423,0.233615,0.306743
3,0.562836,0.605817,0.716051,1.000000,0.407927,0.465003,0.381681,0.423283,0.336042,0.560248,...,0.368120,0.331734,0.497084,0.507649,0.314442,0.510517,0.316763,0.394812,0.276307,0.363090
4,0.356061,0.343116,0.393760,0.407927,1.000000,0.354871,0.220496,0.422171,0.455678,0.353500,...,0.375486,0.489843,0.359303,0.325647,0.263857,0.328353,0.299562,0.441794,0.389199,0.319136


In [0]:
7691   7047

In [0]:
similarity.columns = case_id

In [0]:
eva = pd.read_csv('/content/gdrive/My Drive/Similarity_Evaluation.csv')

In [0]:
case1, case2 = eva['CaseId1'], eva['CaseId2']

In [0]:
sim  = similarity.loc[eva['CaseId1'], eva['CaseId2']]

In [0]:
ev = list([case1, case2])

In [0]:
 a = eva[['CaseId1', 'CaseId2']].values

In [0]:
sim = []
for i,j in a:
  sim.append(similarity.loc[i, j]) 

In [0]:
eva['Similarity'] = np.array(sim)

In [0]:
eva.drop(labels= 'sim', axis = 1, inplace= True)

In [0]:
pd.set_option('display.max_rows', 300)

In [449]:
eva

,CaseId1,CaseId2,Similarity
0,1008,1304,0.636882
1,1008,2740,0.535729
2,1008,5788,0.741867
3,1008,8318,0.319903
4,1049,234,0.796723
5,1049,3905,0.825984
6,1049,4951,0.428932
7,1049,8813,0.380754
8,1056,3270,0.400964
9,1056,3573,0.544777
